In [ ]:
# Code to test the new smote_smfb function

In [ ]:
! pip install pyyaml

In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
import gc
import os

## Modelling related packages
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.datasets import make_classification
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform


In [2]:
from generate_imbalanced_data import generate_imbalanced_data

In [3]:
n_rows = 1000
imbalance_ratio = 0.20  # 10% minority class, 90% majority class
n_columns = 10
n_cols_imp_var = int(n_columns*0.4)  ## no. of top important variables to be chosen
no_of_nn = int ( ( n_rows - (2*imbalance_ratio * n_rows) ) / ( imbalance_ratio * n_rows) ) ## no. of nearest neighbours to get balanced data
flip_y = 0 ## fraction of samples class assigned randomly. Larger values introduce noise in the labels & make the classification task harder. 
class_sep = 0.2 ## Lower the number higher the overlap between boundaries of classes
informative_feature_perc = 0.4 ## Percentage of the informative features

In [4]:
full_data = generate_imbalanced_data(n_rows, imbalance_ratio, n_columns, flip_y, class_sep, informative_feature_perc)

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [5]:
full_data.head(3)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target
0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0


In [18]:
import yaml

with open("/repos/smote_msfb/functions/config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [7]:
sys.path.append('/repos/smote_msfb/functions')

from smote_msfb import smote_msfb 

In [8]:
full_data.iloc[:,:50].head(3)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,target
0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0


In [9]:
full_data['target'].head(3)

0    0
1    0
2    0
Name: target, dtype: int64

In [11]:
y_train = full_data['target']
x_train = full_data.iloc[:,:10]

In [12]:
x_train.head(3)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0


In [19]:
x_train_upd, y_train_upd = smote_msfb(x_train, y_train, config)

<class 'dict'>
LogisticRegression(max_iter=1000)
Total minority samples: 200
Misclassified by bagged classification models: 200
After filtering proba < 0.10: 197 remain
Based on user input the no. of minority samples to be generated : 600
Selected 5 features that account for 80.0% of total mutual information.
No. of irrelevant variables removed for neighbourhood definition: 5
Shape of x_minority_selected : (200, 5)
weights.shape : (5,)
Shape of jaccard_dist matrix : (200, 200)
No. of synthetic samples to be generated: 600
----------- RESAMPLE DATA - NO. OF SYNTHETIC SAMPLES TO BE GENERATED IS HIGHER THAN MISSCLASSIFIED OBSERVATIONS -------------
We are drawing 600 samples from an array of size 197.
Minority Sample for which synthetic obs are generated : 8
Local index for minority sample at 8 in the original data is : 0 
Neighbours of the original data 8 and minority only data_idx 0 which will be used for synthetic obse [  0  96 110  83  30].
Neighbours of the original data 8 and minori